In [2]:
import numpy as np
from sklearn.decomposition import PCA

#PATHS
VIT_PATH = '/home/maria/ProjectionSort/data/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/ProjectionSort/data/hybrid_neural_responses_reduced.npy'
AREAS_PATH = '/home/maria/ProjectionSort/data/brain_area.npy'  


# Load data
vit = np.load(VIT_PATH, allow_pickle=True)['natural_scenes']
R = np.load(NEURAL_PATH)            # shape: (images, neurons)
areas = np.load(AREAS_PATH, allow_pickle=True)         # shape: (neurons,)

# vit shape: (images, 1000)
X = vit.astype(float)

# center
Xc = X - X.mean(axis=0, keepdims=True)

pca = PCA(n_components=3)
pca.fit(Xc)
pc1 = pca.components_[0]   # shape (1000,)

print(pc1.shape)


(1000,)


In [3]:
mid = 400

left  = pc1[:mid]
right = pc1[-mid:][::-1]    # reverse the tail

# correlation of left with NEGATED right
corr = np.corrcoef(left, -right)[0, 1]
print("Mirror anti-symmetry correlation:", corr)

Mirror anti-symmetry correlation: 0.13323707680721192


In [4]:
import numpy as np
from sklearn.decomposition import PCA

X = vit.astype(float)
Xc = X - X.mean(axis=0, keepdims=True)

pca = PCA(n_components=3)
pca.fit(Xc)
pc1 = pca.components_[0]  # shape (1000,)

# symmetry around horizontal axis
mu = pc1.mean()
v = pc1 - mu          # centered eigenvector

pos = v[v > 0].sum()
neg = -v[v < 0].sum()  # negate, so it's positive mass

sym_index = min(pos, neg) / max(pos, neg)
print("Horizontal symmetry index:", sym_index)
print("sum positive:", pos, "sum negative:", neg)


Horizontal symmetry index: 1.0
sum positive: 12.385899355660623 sum negative: 12.385899355660623


In [5]:
print("PC1 mean =", pc1.mean())
print("PC1 min  =", pc1.min())
print("PC1 max  =", pc1.max())
print("PC1 sum  =", pc1.sum())
print("Positive sum =", pc1[pc1 > 0].sum())
print("Negative sum =", pc1[pc1 < 0].sum())

PC1 mean = -9.456972857350365e-06
PC1 min  = -0.08758162171250396
PC1 max  = 0.127738178212524
PC1 sum  = -0.009456972857350365
Positive sum = 12.381785572467678
Negative sum = -12.391242545325028


In [6]:
v = pc1  # no centering here

v_pos = v[v > 0]
v_neg = v[v < 0]

E_pos = np.sum(v_pos**2)
E_neg = np.sum(v_neg**2)

opponent_index = 1 - abs(E_pos - E_neg) / (E_pos + E_neg)
print("Opponent index:", opponent_index)
print("E_pos:", E_pos, "E_neg:", E_neg)


Opponent index: 0.7737097585328523
E_pos: 0.6131451207335741 E_neg: 0.38685487926642625


In [7]:
import numpy as np
from sklearn.decomposition import PCA
import requests
import json
import os

# ============================================================
# 1. PATHS — your data
# ============================================================
VIT_PATH = "/home/maria/ProjectionSort/data/google_vit-base-patch16-224_embeddings_logits.pkl"

# ============================================================
# 2. LOAD ViT logits
# ============================================================
vit = np.load(VIT_PATH, allow_pickle=True)["natural_scenes"]
print("Loaded ViT logits:", vit.shape)

# ============================================================
# 3. PCA → extract PC1
# ============================================================
X = vit.astype(float)
Xc = X - X.mean(axis=0, keepdims=True)   # center across images

pca = PCA(n_components=3)
pca.fit(Xc)

pc1 = pca.components_[0]       # shape: (1000,)
print("PC1 shape:", pc1.shape)

# ============================================================
# 4. Download ImageNet 1k class index → labels mapping
# ============================================================
URL = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
# this is the standard mapping file used by many libraries
resp = requests.get(URL)
resp.raise_for_status()
class_idx = resp.json()   # dict of str index to [synset_id, human_readable_label]

# Convert to int->label dict
idx_to_label = {int(k): v[1] for k, v in class_idx.items()}
print("Loaded ImageNet labels:", len(idx_to_label))

# ============================================================
# 5. Extract top POSITIVE & NEGATIVE classes from PC1
# ============================================================
k = 20   # how many to print

top_pos_idx = np.argsort(pc1)[-k:][::-1]
top_neg_idx = np.argsort(pc1)[:k]

print("\n======================")
print("Top POSITIVE PC1 classes")
print("======================")
for i in top_pos_idx:
    print(f"{i:>4d} | {pc1[i]: .5f} | {idx_to_label.get(i, 'UNKNOWN')}")

print("\n======================")
print("Top NEGATIVE PC1 classes")
print("======================")
for i in top_neg_idx:
    print(f"{i:>4d} | {pc1[i]: .5f} | {idx_to_label.get(i, 'UNKNOWN')}")


Loaded ViT logits: (118, 1000)
PC1 shape: (1000,)
Loaded ImageNet labels: 1000

Top POSITIVE PC1 classes
 291 |  0.12774 | lion
 274 |  0.12317 | dhole
 288 |  0.12011 | leopard
 293 |  0.11944 | cheetah
 271 |  0.11403 | red_wolf
 292 |  0.10888 | tiger
 290 |  0.10688 | jaguar
 272 |  0.10602 | coyote
 269 |  0.10511 | timber_wolf
 280 |  0.10365 | grey_fox
 286 |  0.10362 | cougar
 275 |  0.10024 | African_hunting_dog
 276 |  0.09944 | hyena
 294 |  0.09882 | brown_bear
 289 |  0.09834 | snow_leopard
 273 |  0.09156 | dingo
 270 |  0.09149 | white_wolf
 287 |  0.09018 | lynx
 278 |  0.08686 | kit_fox
 371 |  0.08488 | patas

Top NEGATIVE PC1 classes
 421 | -0.08758 | bannister
 879 | -0.07011 | umbrella
 412 | -0.06883 | ashcan
 716 | -0.06246 | picket_fence
 703 | -0.06210 | park_bench
 637 | -0.06106 | mailbox
 818 | -0.06027 | spotlight
 883 | -0.05732 | vase
 489 | -0.05713 | chainlink_fence
 898 | -0.05627 | water_bottle
 880 | -0.05572 | unicycle
 733 | -0.05510 | pole
 706 | 

In [8]:
import numpy as np
import requests
import json

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim

# -------------------------------------------------
# Paths
# -------------------------------------------------
VIT_PATH = "/home/maria/ProjectionSort/data/google_vit-base-patch16-224_embeddings_logits.pkl"

# -------------------------------------------------
# Load ViT logits
# -------------------------------------------------
vit = np.load(VIT_PATH, allow_pickle=True)["natural_scenes"]  # (n_images, 1000)
print("Loaded ViT logits:", vit.shape)

n_images, n_classes = vit.shape

# -------------------------------------------------
# Download ImageNet labels (same as before)
# -------------------------------------------------
URL = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
resp = requests.get(URL)
resp.raise_for_status()
class_idx = resp.json()   # { "0": ["n01440764", "tench"], ... }
idx_to_label = {int(k): v[1] for k, v in class_idx.items()}

# -------------------------------------------------
# Use your discovered extremes as class sets
# (You can tweak / expand these if you like)
# -------------------------------------------------
pos_classes = [
    291, 274, 288, 293, 271, 292, 290, 272, 269, 280,
    286, 275, 276, 294, 289, 273, 270, 287, 278, 371,
]

neg_classes = [
    421, 879, 412, 716, 703, 637, 818, 883, 489, 898,
    880, 733, 706, 523, 738, 616, 448, 696, 704, 506,
]

pos_set = set(pos_classes)
neg_set = set(neg_classes)

# -------------------------------------------------
# Build a clean binary dataset
# -------------------------------------------------
X_list = []
y_list = []

top1 = np.argmax(vit, axis=1)

for i in range(n_images):
    c = int(top1[i])
    if c in pos_set:
        X_list.append(vit[i])
        y_list.append(1)
    elif c in neg_set:
        X_list.append(vit[i])
        y_list.append(0)
    else:
        continue  # ignore all other images

X = np.stack(X_list, axis=0)  # (N, 1000)
y = np.array(y_list)          # (N,)

print("Binary dataset shape:", X.shape, y.shape)
print("Class balance: #neg =", (y == 0).sum(), " #pos =", (y == 1).sum())

# -------------------------------------------------
# Train/test split
# -------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Convert to torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_t  = torch.tensor(X_test,  dtype=torch.float32).to(device)
y_test_t  = torch.tensor(y_test,  dtype=torch.float32).to(device)

# -------------------------------------------------
# Single "animal neuron" model
# -------------------------------------------------
class AnimalNeuron(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.linear = nn.Linear(in_dim, 1)  # w, b

    def forward(self, x):
        # returns probability in (0,1)
        return torch.sigmoid(self.linear(x)).squeeze(-1)

model = AnimalNeuron(n_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

# -------------------------------------------------
# Training loop (tiny, just a few epochs)
# -------------------------------------------------
num_epochs = 20
batch_size = 64

def batches(X, y, bs):
    n = X.shape[0]
    idx = torch.randperm(n)
    for start in range(0, n, bs):
        end = min(start + bs, n)
        sel = idx[start:end]
        yield X[sel], y[sel]

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for xb, yb in batches(X_train_t, y_train_t, batch_size):
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * xb.size(0)

    epoch_loss /= X_train_t.shape[0]

    # quick evaluation
    model.eval()
    with torch.no_grad():
        preds_test = model(X_test_t)
        pred_labels = (preds_test > 0.5).float()
        acc = (pred_labels == y_test_t).float().mean().item()

    print(f"Epoch {epoch+1:2d} | loss {epoch_loss:.4f} | test acc {acc:.3f}")


Loaded ViT logits: (118, 1000)
Binary dataset shape: (41, 1000) (41,)
Class balance: #neg = 24  #pos = 17
Epoch  1 | loss 0.6976 | test acc 0.667
Epoch  2 | loss 0.4924 | test acc 0.889
Epoch  3 | loss 0.3445 | test acc 1.000
Epoch  4 | loss 0.2428 | test acc 1.000
Epoch  5 | loss 0.1742 | test acc 1.000
Epoch  6 | loss 0.1279 | test acc 1.000
Epoch  7 | loss 0.0962 | test acc 1.000
Epoch  8 | loss 0.0740 | test acc 1.000
Epoch  9 | loss 0.0581 | test acc 1.000
Epoch 10 | loss 0.0464 | test acc 1.000
Epoch 11 | loss 0.0376 | test acc 1.000
Epoch 12 | loss 0.0310 | test acc 1.000
Epoch 13 | loss 0.0258 | test acc 1.000
Epoch 14 | loss 0.0218 | test acc 1.000
Epoch 15 | loss 0.0186 | test acc 1.000
Epoch 16 | loss 0.0161 | test acc 1.000
Epoch 17 | loss 0.0141 | test acc 1.000
Epoch 18 | loss 0.0124 | test acc 1.000
Epoch 19 | loss 0.0110 | test acc 1.000
Epoch 20 | loss 0.0099 | test acc 1.000


In [9]:
import torch.nn.functional as F

def run_dynamics(x_np, model, alpha=0.9, T=10):
    """
    x_np: numpy array shape (1000,)
    model: trained AnimalNeuron
    """
    model.eval()
    with torch.no_grad():
        w = model.linear.weight.detach().cpu()[0]  # (1000,)
        b = model.linear.bias.detach().cpu().item()

    x = torch.tensor(x_np, dtype=torch.float32)
    h = torch.tensor(0.0)

    traj = []
    for t in range(T):
        h_in = torch.dot(w, x) + b + alpha * h
        h = torch.sigmoid(h_in)
        traj.append(h.item())

    return traj

# Example: pick an image that ViT thinks is a lion
example_idx = np.where(top1 == 291)[0][0]  # top1 from before
traj_lion = run_dynamics(vit[example_idx], model, alpha=0.9, T=10)

print("Dynamics for a 'lion' image:")
for t, h_t in enumerate(traj_lion):
    print(f"t={t}  h={h_t:.3f}")


Dynamics for a 'lion' image:
t=0  h=0.992
t=1  h=0.997
t=2  h=0.997
t=3  h=0.997
t=4  h=0.997
t=5  h=0.997
t=6  h=0.997
t=7  h=0.997
t=8  h=0.997
t=9  h=0.997
